In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
pd.set_option('display.max_columns', None)
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#Importing the data
final_df=pd.DataFrame()
a_df=pd.DataFrame()
b_df=pd.DataFrame()
c_df=pd.DataFrame()

for file_n in os.listdir('listings'):
    if file_n[0]=='.':
        continue
    else:
        add_df = pd.read_csv(f'listings/{file_n}')
        if len(add_df.columns)==106:
            final_df = pd.concat([final_df, add_df], axis=0)
        elif len(add_df.columns)==96:
            a_df = pd.concat([a_df, add_df], axis=0)
        elif len(add_df.columns)==95:
            b_df = pd.concat([b_df, add_df], axis=0)
        else:
            c_df = pd.concat([c_df, add_df], axis=0)

/Users/sherryduong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/sherryduong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/sherryduong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
#These are the columns not present in 2018 reports. They will be dropped.
not_in_2018 = ['minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights',
       'minimum_nights_avg_ntm','maximum_nights_avg_ntm', 'number_of_reviews_ltm', 
       'calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms', 
       'calculated_host_listings_count_shared_rooms']
sf_df = final_df.drop(columns=not_in_2018)
sf_df = pd.concat([sf_df, a_df], axis=0)

#drop one column not in 2017
sf_df = sf_df.drop(columns=['is_business_travel_ready'])
sf_df = pd.concat([sf_df, b_df], axis=0)

#drop 3 columns not in 2016 & Prior
sf_df = sf_df.drop(columns=['access', 'interaction', 'house_rules'])
sf_df = pd.concat([sf_df, c_df], axis=0)

/Users/sherryduong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [4]:
pd.set_option('display.max_rows', 20)
#Converting Existing Data Columns to Dates
sf_df['last_scraped'] = pd.to_datetime(sf_df['last_scraped'])
sf_df['host_since'] = pd.to_datetime(sf_df['host_since'])
#Adding new date features
sf_df['year'] = sf_df['last_scraped'].transform(lambda x: x.year)
sf_df['month-year'] = sf_df['last_scraped'].transform(lambda x: f'{x.month} {x.year}')
sf_df['month'] = sf_df['last_scraped'].transform(lambda x: x.month)
sf_df['day_of_week'] = sf_df['last_scraped'].transform(lambda x: x.dt.dayofweek)
sf_df['day'] = sf_df['last_scraped'].transform(lambda x: x.day)

#Converting All Price Related Columns from Objects to Floats
sf_df['price'] = sf_df['price'].transform(lambda x: float(x.replace(',', '').replace('$', '')))
sf_df['extra_people'] = sf_df['extra_people'].transform(lambda x: float(x.replace(',', '').replace('$', '')))

#Fill NaNs in fee columns with 0's, because no additional fee, then convert
sf_df[['security_deposit','cleaning_fee']] = sf_df[['security_deposit','cleaning_fee']].fillna(int(0))
sf_df['security_deposit'] = sf_df['security_deposit'].transform(lambda x: 0 if x==int(0) else float(x.replace(',', '').replace('$', '')))
sf_df['cleaning_fee'] = sf_df['cleaning_fee'].transform(lambda x: 0 if x==int(0) else float(x.replace(',', '').replace('$', '')))

#Dropping columns that have over 75% null
over_70_null = sf_df.columns[sf_df.isnull().sum()/len(sf_df) > 0.70]
sf_df = sf_df.drop(columns=over_70_null)

#Removing Outliers (0 & over 2000 daily rate)
outlier_thresh=2000
sf_df = sf_df.loc[~((sf_df['price'] == 0) | (sf_df['price'] > outlier_thresh))]

#Filling the NaNs in beds, bathrooms, bedrooms
#Assume that if the tenant has the full apartment, they have a bathroom/bedroom
cond1= (sf_df['room_type']=='Entire home/apt')
sf_df.loc[cond1 & (sf_df['beds']==0), sf_df.columns=='beds'] = 1
sf_df.loc[cond1 & (sf_df['bathrooms']==0), sf_df.columns=='bathrooms'] = 1
#For the rest, just fill na
sf_df[['beds','bathrooms', 'bedrooms']] = sf_df[['beds','bathrooms', 'bedrooms']].fillna(0)

#For review scores, let's just fillin with the average
review_lst = ['review_scores_accuracy', 'review_scores_checkin',
       'review_scores_cleanliness', 'review_scores_communication',
       'review_scores_location', 'review_scores_rating', 'review_scores_value']
for review in review_lst:
    sf_df[review] = sf_df[review].fillna(sf_df[review].mean())

In [5]:
train_df2, test = train_test_split(sf_df, test_size = 0.3)

In [6]:
#Adding in all feature engineering for best performing model with Random Forest
train_df2['amenities'] = train_df2['amenities'].fillna('0')
train_df2['num_amenities'] = train_df2['amenities'].transform(lambda x: len(x.split(',')))
train_df2[['space','summary','description','name']] = train_df2[['space','summary','description','name']].fillna(0)
train_df2['len_space'] = train_df2['space'].transform(lambda x: 0 if x==0 else len(x))
train_df2['len_summary'] = train_df2['summary'].transform(lambda x: 0 if x==0 else len(x))
train_df2['len_description'] = train_df2['description'].transform(lambda x: 0 if x==0 else len(x))
train_df2['len_name'] = train_df2['name'].transform(lambda x: 0 if x==0 else len(x))

/Users/sherryduong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/sherryduong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sherryduong/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [36]:
features=['accommodates','bathrooms', 'bed_type','bedrooms', 'beds','cleaning_fee',
          'extra_people', 'num_amenities','neighbourhood_cleansed','property_type',
          'review_scores_cleanliness','len_space', 'len_summary', 'len_description',
          'len_name', 'review_scores_rating', 'room_type', 'security_deposit', 'month', 
          'year']
X7 = train_df2[features]
y7 = train_df2['price'].apply(np.log)

X7 = pd.get_dummies(X7, columns=['bed_type','neighbourhood_cleansed','property_type',
                           'room_type','month'])

In [37]:
X_train, X_val, y_train, y_val = train_test_split(X7, y7, test_size=0.3)

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [21]:
X_train = tf.keras.utils.normalize(X_train, axis=1).values
y_train = np.array(y_train)

In [22]:
model = Sequential()

#Input layer
model.add(Dense(1))
model.add(Activation("relu"))

#Second layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

#Output layer
model.add(Dense(1))
model.add(Activation('relu'))

model.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['MeanSquaredError'])
model.fit(X,y, batch_size=32, epochs=10, validation_split=0.1)

Train on 173524 samples, validate on 19281 samples
Epoch 1/10
173524/173524 [==============================] - 27s 153us/sample - loss: 0.9398 - MeanSquaredError: 0.9398 - val_loss: 0.3804 - val_MeanSquaredError: 0.3804
Epoch 2/10
173524/173524 [==============================] - 8s 46us/sample - loss: 0.3658 - MeanSquaredError: 0.3658 - val_loss: 0.3489 - val_MeanSquaredError: 0.3489
Epoch 3/10
173524/173524 [==============================] - 9s 52us/sample - loss: 0.3122 - MeanSquaredError: 0.3122 - val_loss: 0.2744 - val_MeanSquaredError: 0.2744
Epoch 4/10
173524/173524 [==============================] - 9s 53us/sample - loss: 0.2519 - MeanSquaredError: 0.2519 - val_loss: 0.2359 - val_MeanSquaredError: 0.2359
Epoch 5/10
173524/173524 [==============================] - 8s 45us/sample - loss: 0.2346 - MeanSquaredError: 0.2346 - val_loss: 0.2274 - val_MeanSquaredError: 0.2274
Epoch 6/10
173524/173524 [==============================] - 8s 47us/sample - loss: 0.2261 - MeanSquaredError: 0.

In [23]:
X_val = tf.keras.utils.normalize(X_val, axis=1).values
y_val = np.array(y_val)
preds = model.predict(X_val) 

In [28]:
rmse = np.exp(np.sqrt(mean_squared_error(y_val, preds)))
r2 = r2_score(y_val, preds)
print(rmse, r2)

1.5694033226163753 0.5781447619611689


## Add More Layers

In [33]:
model1 = Sequential()

#Input layer
model1.add(Dense(1))
model1.add(Activation("relu"))

#Second layer
model1.add(Flatten())
model1.add(Dense(64))
model1.add(Activation('relu'))

#Third layer
model1.add(Flatten())
model1.add(Dense(64))
model1.add(Activation('relu'))

#Fourth layer
model1.add(Flatten())
model1.add(Dense(64))
model1.add(Activation('relu'))

#Output layer
model1.add(Dense(1))
model1.add(Activation('relu'))

model1.compile(loss='mean_squared_error',
             optimizer='adam',
             metrics=['MeanSquaredError'])
model1.fit(X,y, epochs=10, validation_split=0.1)

Train on 173524 samples, validate on 19281 samples
Epoch 1/10
173524/173524 [==============================] - 9s 51us/sample - loss: 1.0905 - MeanSquaredError: 1.0905 - val_loss: 0.2340 - val_MeanSquaredError: 0.2340 - MeanSquaredError
Epoch 2/10
173524/173524 [==============================] - 8s 46us/sample - loss: 0.2337 - MeanSquaredError: 0.2337 - val_loss: 0.2160 - val_MeanSquaredError: 0.2160
Epoch 3/10
173524/173524 [==============================] - 8s 47us/sample - loss: 0.2176 - MeanSquaredError: 0.2176 - val_loss: 0.2005 - val_MeanSquaredError: 0.2005
Epoch 4/10
173524/173524 [==============================] - 8s 48us/sample - loss: 0.2091 - MeanSquaredError: 0.2091 - val_loss: 0.1947 - val_MeanSquaredError: 0.1947
Epoch 5/10
173524/173524 [==============================] - 8s 47us/sample - loss: 0.2049 - MeanSquaredError: 0.2049 - val_loss: 0.1951 - val_MeanSquaredError: 0.1951
Epoch 6/10
173524/173524 [==============================] - 8s 45us/sample - loss: 0.2022 - Mea

In [35]:
preds = model1.predict(X_val)
rmse = np.exp(np.sqrt(mean_squared_error(y_val, preds)))
r2 = r2_score(y_val, preds)
print(rmse, r2)

1.5482691700609095 0.6031435729747893
